In [1]:
#XGBoost on complete dataset - normalized data - no oversampling

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel('/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/df_complete.xlsx')

In [4]:
df = df.drop(columns=['Unnamed: 0', 'TotalPrice'])

In [5]:
df.head()

,Customer_ID,recency,frequency,monetary,Top_10%_spend
0,12346,0.439678,0.049020,0.001059,1
1,12347,0.005362,0.004902,0.003782,1
2,12348,0.195710,0.000000,0.000628,0
3,12349,0.112601,0.009804,0.007642,0
4,12352,0.026810,0.004902,0.000976,0


In [6]:
X = df[['recency', 'frequency', 'monetary']]
y = df['Top_10%_spend']

In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBClassifier(random_state = 123, use_label_encoder=False, eval_metric = 'error')
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Lowest RMSE:  0.28678576113117793


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)

In [8]:
import xgboost as xgb
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, random_state = 123, use_label_encoder=False, colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 500)

In [9]:
clf_xgb.fit(X_train, 
            y_train,
            verbose = True,
            early_stopping_rounds = 10,
            eval_metric = 'aucpr',
            eval_set = [[X_test, y_test]])

[0]	validation_0-aucpr:0.71381
[1]	validation_0-aucpr:0.71128
[2]	validation_0-aucpr:0.71555
[3]	validation_0-aucpr:0.71628
[4]	validation_0-aucpr:0.71509
[5]	validation_0-aucpr:0.71530
[6]	validation_0-aucpr:0.71654
[7]	validation_0-aucpr:0.71654
[8]	validation_0-aucpr:0.71555
[9]	validation_0-aucpr:0.71594
[10]	validation_0-aucpr:0.74360
[11]	validation_0-aucpr:0.74380
[12]	validation_0-aucpr:0.74384
[13]	validation_0-aucpr:0.74470
[14]	validation_0-aucpr:0.74514
[15]	validation_0-aucpr:0.74500
[16]	validation_0-aucpr:0.74478
[17]	validation_0-aucpr:0.74686
[18]	validation_0-aucpr:0.74704
[19]	validation_0-aucpr:0.74768
[20]	validation_0-aucpr:0.74819
[21]	validation_0-aucpr:0.74770
[22]	validation_0-aucpr:0.74863
[23]	validation_0-aucpr:0.74895
[24]	validation_0-aucpr:0.74877
[25]	validation_0-aucpr:0.74877
[26]	validation_0-aucpr:0.74886
[27]	validation_0-aucpr:0.74885
[28]	validation_0-aucpr:0.74918
[29]	validation_0-aucpr:0.74944
[30]	validation_0-aucpr:0.74964
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [10]:
xgb_preds = clf_xgb.predict(X_test)

In [11]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, xgb_preds)
cnf_matrix

array([[707,  15],
       [ 46,  64]])

In [12]:
print("Accuracy:",metrics.accuracy_score(y_test, xgb_preds))
print("Precision:",metrics.precision_score(y_test, xgb_preds))
print("Recall:",metrics.recall_score(y_test, xgb_preds))

Accuracy: 0.9266826923076923
Precision: 0.810126582278481
Recall: 0.5818181818181818


In [13]:
y_pred_proba = clf_xgb.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)

In [14]:
print(auc)

0.9078443716947873


In [15]:
f1_xgb = metrics.f1_score(y_test, xgb_preds)
print(f1_xgb)

0.6772486772486772


In [ ]:
# XGBoost on complete dataset - normalized data - oversampling

In [16]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [17]:
ros = RandomOverSampler(random_state=123)
X_train_ros, y_train_ros= ros.fit_resample(X_train, y_train)

In [18]:
import xgboost as xgb
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, random_state = 123, use_label_encoder=False, colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 500)
test_model = clf_xgb.fit(X_train_ros, y_train_ros)
test_prediction = test_model.predict(X_test)

[13:03:08] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [19]:
cnf_matrix = metrics.confusion_matrix(y_test, test_prediction)
cnf_matrix

array([[631,  91],
       [ 20,  90]])

In [20]:
print("Accuracy:",metrics.accuracy_score(y_test, test_prediction))
print("Precision:",metrics.precision_score(y_test, test_prediction))
print("Recall:",metrics.recall_score(y_test, test_prediction))

Accuracy: 0.8665865384615384
Precision: 0.4972375690607735
Recall: 0.8181818181818182


In [21]:
y_pred_proba_os = clf_xgb.predict_proba(X_test)[::,1]
auc_os = metrics.roc_auc_score(y_test, y_pred_proba_os)

In [22]:
print(auc_os)

0.9182825484764543


In [23]:
f1_xgb_os = metrics.f1_score(y_test, test_prediction)
print(f1_xgb_os)

0.6185567010309279


In [24]:
#XGBoost on dataset with additional features - normalized data - not oversampled

In [25]:
df2 = pd.read_excel('/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/df_add_features.xlsx')

In [26]:
df2.head()

,Unnamed: 0,Customer_ID,recency,frequency,monetary,TotalPrice,Top_10%_spend,y1 average quantity,y1 average spend,Country,Customer ID,cluster
0,0,12346,0.439678,0.049020,0.001059,77183.60,1,2.121212,11.298788,United Kingdom,12346,0
1,1,12347,0.005362,0.004902,0.003782,4310.00,1,11.661972,18.638310,Iceland,12347,0
2,2,12348,0.195710,0.000000,0.000628,1797.24,0,18.650000,11.108000,Finland,12348,0
3,3,12349,0.112601,0.009804,0.007642,1757.55,0,9.735294,26.187647,Italy,12349,0
4,4,12352,0.026810,0.004902,0.000976,2506.04,0,10.444444,19.100000,Norway,12352,0


In [36]:
df2['Country'].value_counts()

United Kingdom          2539
Germany                   55
France                    39
Spain                     14
Portugal                  13
Belgium                   12
Switzerland               12
Netherlands                9
Austria                    8
Denmark                    8
Australia                  8
Italy                      8
Channel Islands            7
Sweden                     5
Finland                    5
Japan                      4
Cyprus                     4
EIRE                       3
Greece                     3
Bahrain                    2
Norway                     2
Poland                     2
United Arab Emirates       2
Unspecified                2
USA                        1
Israel                     1
Iceland                    1
Malta                      1
Lithuania                  1
Singapore                  1
Name: Country, dtype: int64

In [37]:
X2 = df2[['recency', 'frequency', 'monetary', 'y1 average quantity', 'y1 average spend']]
y2 = df2['Top_10%_spend']

In [34]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBClassifier(random_state = 123, use_label_encoder=False, eval_metric = 'error')
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
clf.fit(X2, y2)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Lowest RMSE:  0.28741674369779335


In [38]:
from sklearn.model_selection import train_test_split
X_train2,X_test2,y_train2,y_test2=train_test_split(X2,y2,test_size=0.3,random_state=123)

In [39]:
import xgboost as xgb
clf_xgb2 = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, random_state = 123, use_label_encoder=False, colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 100)

In [40]:
clf_xgb2.fit(X_train2, 
            y_train2,
            verbose = True,
            early_stopping_rounds = 10,
            eval_metric = 'aucpr',
            eval_set = [[X_test2, y_test2]])

[0]	validation_0-aucpr:0.68860
[1]	validation_0-aucpr:0.73210
[2]	validation_0-aucpr:0.74107
[3]	validation_0-aucpr:0.73825
[4]	validation_0-aucpr:0.73655
[5]	validation_0-aucpr:0.73695
[6]	validation_0-aucpr:0.74737
[7]	validation_0-aucpr:0.74636
[8]	validation_0-aucpr:0.74703
[9]	validation_0-aucpr:0.74668
[10]	validation_0-aucpr:0.74708
[11]	validation_0-aucpr:0.74506
[12]	validation_0-aucpr:0.74573
[13]	validation_0-aucpr:0.74617
[14]	validation_0-aucpr:0.74457
[15]	validation_0-aucpr:0.74295


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=123,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [41]:
xgb_preds2 = clf_xgb2.predict(X_test2)

In [42]:
from sklearn import metrics
cnf_matrix2 = metrics.confusion_matrix(y_test2, xgb_preds2)
cnf_matrix2

array([[707,  15],
       [ 45,  65]])

In [43]:
print("Accuracy:",metrics.accuracy_score(y_test2, xgb_preds2))
print("Precision:",metrics.precision_score(y_test2, xgb_preds2))
print("Recall:",metrics.recall_score(y_test2, xgb_preds2))

Accuracy: 0.9278846153846154
Precision: 0.8125
Recall: 0.5909090909090909


In [44]:
y_pred_proba2 = clf_xgb2.predict_proba(X_test2)[::,1]
auc2 = metrics.roc_auc_score(y_test2, y_pred_proba2)

In [45]:
print(auc2)

0.9093112566104256


In [46]:
f1_xgb2 = metrics.f1_score(y_test2, xgb_preds2)
print(f1_xgb2)

0.6842105263157896


In [ ]:
#XGBoost on dataset with additional features - normalized data - oversampled

In [50]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [51]:
ros = RandomOverSampler(random_state=123)
X_train_ros2, y_train_ros2= ros.fit_resample(X_train2, y_train2)

In [52]:
import xgboost as xgb
clf_xgb2 = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, random_state = 123, use_label_encoder=False, colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 100)
test_model2 = clf_xgb2.fit(X_train_ros2, y_train_ros2)
test_prediction2 = test_model2.predict(X_test2)

[11:57:24] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [53]:
cnf_matrix = metrics.confusion_matrix(y_test2, test_prediction2)
cnf_matrix

array([[634,  88],
       [ 20,  90]])

In [54]:
print("Accuracy:",metrics.accuracy_score(y_test2, test_prediction2))
print("Precision:",metrics.precision_score(y_test2, test_prediction2))
print("Recall:",metrics.recall_score(y_test2, test_prediction2))

Accuracy: 0.8701923076923077
Precision: 0.5056179775280899
Recall: 0.8181818181818182


In [55]:
y_pred_proba2_os = clf_xgb2.predict_proba(X_test)[::,1]
auc_2os = metrics.roc_auc_score(y_test, y_pred_proba2_os)

In [56]:
print(auc_2os)

0.9177159405691261


In [57]:
f1_xgb_2os = metrics.f1_score(y_test2, test_prediction2)
print(f1_xgb_2os)

0.6250000000000001
